<a href="https://colab.research.google.com/github/tmtakashi/machine_learning_notebooks/blob/master/word2vec_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
% cd /content/drive/My Drive/Colab Notebooks/fusic/word2vec

/content/drive/My Drive/Colab Notebooks/fusic/word2vec


# word2vecまとめ

## 自然言語処理の手法
- シソーラス
  - 類似語をつなぎ合わせて単語ネットワークを作る
  - 最も有名なシソーラス：WordNet
  - 問題点：
    - 時代の変化に対応しにくい
    - 高人的コスト
    - 大まかにグループ化されているため細かいニュアンスが表現しにくいｚ
    
- カウントベース
    - 前提：「単語の意味は、周囲の単語によって形成される」（分布仮説）
    - 単語の共起行列（他の単語と隣り合う頻度を表す）を特異値分解（次元削減）→　単語の分散表現を得る。
    - 問題点：コーパス全体（英語だと約100万語）に対して特異値分解をすると計算量がエグい。
    
- 推論ベース
    - 学習データの一部を使って逐次的にニューラルネットワークで処理。
    - word2vec
      - Skip-gramとCBOW    
- 最近ではGloveというカウントベースと推論ベースを組み合わせたモデルも出てきている



## 推論ベースの手法の概要
- 周囲の単語が与えられたときに、目的の単語を予測 or vice versa
    - 推論問題を繰り返し解いて学習
    - コンテキスト→モデル→確率分布（予測）
### ニューラルネットワークにおける単語の処理方法
- 単語→単語ID→one-hot表現
    - ex. "You say goodbye and I say hello."の一文をコーパスとして捉える
        - 単語ID => 0 ~ 6
        
- 重み行列の各行を抽出している　→　各行が各単語のベクトル表現になっている

## Skip-gram
- 入力：中心語（中央の単語)
- 出力：文脈語（周囲の単語）

- 例文：I love green eggs and ham.
    - ウインドウサイズ1を仮定、（文脈語、中心語）のペアに分解
        - ([I, green], love), ([love, eggs], green), ([green, and], eggs)
    - 負例（実際の文章に含まれる組み合わせとはことなるもの）を生成
        - (love, Sam), (love, zebra), (green, thing)
    - 正例と負例を生成
        - ((love, I), 1), ((love, green), 1), ..., ((love, Sam), 0), ((love, zebra), 0), ...
        
- ネットワークを学習させEmbedding層の重みを得れば単語IDから単語のベクトル表現の重みを得られる。

### Embedding層
- 各行を抜き出すだけでよいのに、行列の掛け算をするのは非効率
- 重みパラメータから単語IDに該当する行を抜き出す層

In [0]:
import numpy as np

from keras.layers import Dot, Dense, Reshape, Embedding, Input
from keras.models import Model

vocab_size = 5000
embed_size = 300

# http://cookie-box.hatenablog.com/entry/2018/10/14/184801
class SkipGramDiscriminator():
  def __init__(self, vocab_size, embed_size):
    self.vocab_size = vocab_size #  語彙数
    self.embed_size = embed_size # 埋め込み次元数
  def create_model(self):
    # 中心語ID => 中心語数値ベクトル表現
    x0 = Input(shape=(1, ))
    y0 = Embedding(self.vocab_size, self.embed_size,
                  embeddings_initializer='glorot_uniform')(x0)
    y0 = Reshape((self.embed_size, ))(y0)
    self.word_embedder = Model(x0, y0)
    # 文脈語ID => 文脈語数値ベクトル表現（実装上は左右いずれかの単語）
    x1 = Input(shape=(1, )) 
    y1 = Embedding(self.vocab_size, self.embed_size,
                  embeddings_initializer='glorot_uniform')(x1)
    y1 = Reshape((self.embed_size, ))(y1)
    self.context_embedder = Model(x1, y1)
    # 内積 => ロジスティック回帰
    y = Dot(axes=-1)([y0, y1])
    y = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(y)
    self.discriminator = Model(inputs=[x0, x1], outputs=y)
    self.discriminator.compile(loss='mean_squared_error', optimizer='adam')
    self.discriminator.summary()
    
from keras.preprocessing.text import * 
from keras.preprocessing.sequence import skipgrams

text = "I love green eggs and ham ."

# 各単語を整数IDにマッピングする辞書を作成
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

wids = [word2id[w] for w in text_to_word_sequence(text)]
pairs, labels = skipgrams(wids, len(word2id), window_size=1)

print(len(pairs), len(labels))
for i in range(10):
  print("({:s} ({:d}), {:s}({:d})) -> {:d}".format(
            id2word[pairs[i][0]], pairs[i][0],
            id2word[pairs[i][1]], pairs[i][1],
            labels[i]))
  
sg = SkipGramDiscriminator(6, 3)
sg.create_model()
x0_samples = np.array([[1], [4], [1], [4], [2]]) # 中心語： love,and,love,and,green
x1_samples = np.array([[0], [5], [2], [2], [2]]) # 文脈語： i,ham,green,green,green
y_samples = sg.discriminator.predict([x0_samples, x1_samples])
print(y_samples) # 中心語と文脈語のペアであるかどうかの判定結果（学習まだ）

print('中心語の数値ベクトル表現は中心語の Embedding 層の重みそのもの\n', sg.word_embedder.get_weights())

# IDから数値ベクトル表現を取り出せることの確認
print('i の数値ベクトル表現: ', sg.word_embedder.predict([[0]]))
print('love の数値ベクトル表現', sg.word_embedder.predict([[1]])) 

20 20
(ham (6), green(3)) -> 0
(i (1), love(2)) -> 1
(love (2), green(3)) -> 1
(eggs (4), love(2)) -> 0
(and (5), i(1)) -> 0
(and (5), i(1)) -> 0
(and (5), ham(6)) -> 1
(green (3), eggs(4)) -> 1
(green (3), love(2)) -> 1
(i (1), i(1)) -> 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 3)         18          input_6[0][0]                    
___________________________________________________________________

## CBOW(continuous bag-of-words)
- 入力：文脈語（周囲の単語）
- 出力：中心語（中央の単語）
    - このモデルができるだけ正確な推測ができるように訓練
- Embedding層→Lambda層（分散表現の平均値を計算）→Dense層→Softmax層
- Embedding層の重みが結果として得られる。

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Lambda, Embedding
import keras.backend as K

vocab_size = 5000
embed_size = 300
window_size = 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embed_size,
                                        embeddings_initializer='glorot_uniform',
                                        input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size, )))
model.add(Dense(vocab_size, kernel_initializer='glorot_uniform', activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Using TensorFlow backend.


## 分散表現の抽出
- 分類問題を解くことが目的ではない。
- 単語を低次元の分散表現に変換する重み行列に興味がある。

## gensim
- word2vecの実装を含むライブラリ
- text8 ：Wikipediaの文章から作られたコーパス

In [0]:
from gensim.models import word2vec

sentences = word2vec.Text8Corpus("text8", 50) # 50単語ずつの文に分割
model = word2vec.Word2Vec(sentences, size=300, min_count=30)

model.init_sims(replace=True) #　正規化（メモリの消費量を大幅に削減できるらしい）
model.save("word2vec_gensim.bin")
model = word2vec.Word2Vec.load("word2vec_gensim.bin")

In [0]:
# 単語一覧取得
len(list(model.wv.vocab.keys()))

25097

In [0]:
# 単語の分散表現取得
model.wv['microsoft']

array([-0.01450746,  0.02820498, -0.00613089, -0.1292292 , -0.00107782,
       -0.03602763, -0.01437078, -0.04320436,  0.01124886, -0.01775628,
        0.01124919, -0.03617769,  0.0851421 , -0.02290596, -0.06446875,
       -0.09365334,  0.01692839,  0.02593309, -0.06995475,  0.02601767,
        0.04263943, -0.15761417,  0.00831248,  0.01263973,  0.04029242,
        0.0277396 , -0.1320366 , -0.05115852,  0.03440702, -0.00313816,
       -0.03922254,  0.01206683,  0.02870189, -0.040955  , -0.01835576,
        0.08676125,  0.00474388,  0.01969292, -0.00248312, -0.05054165,
       -0.01282372, -0.04106919, -0.00888264,  0.01175106, -0.01316832,
        0.08711268,  0.12674089, -0.16148517, -0.08232152, -0.0202497 ,
        0.12008891, -0.02567862, -0.04227788,  0.08244476, -0.02162206,
       -0.13875192,  0.01047679,  0.05519961,  0.12552491,  0.0270695 ,
        0.05264525,  0.0920076 , -0.01546904,  0.09309576,  0.01681585,
       -0.05215343,  0.02041112, -0.08895825,  0.02945577,  0.04

In [0]:
# 類似した語を表示（コサイン類似度）
model.wv.most_similar('man')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.6701514720916748),
 ('girl', 0.5859889388084412),
 ('creature', 0.5346399545669556),
 ('boy', 0.4951860010623932),
 ('person', 0.48739463090896606),
 ('men', 0.4811897873878479),
 ('stranger', 0.4765658974647522),
 ('thief', 0.4699924886226654),
 ('ahab', 0.46971046924591064),
 ('gentleman', 0.4658960998058319)]

In [0]:
model.wv.most_similar('google')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('yahoo', 0.7883809804916382),
 ('wiki', 0.7523242235183716),
 ('newsgroup', 0.7492305040359497),
 ('https', 0.7419716119766235),
 ('usenet', 0.7396144866943359),
 ('ftp', 0.7395017147064209),
 ('msn', 0.7351704835891724),
 ('faq', 0.7306565046310425),
 ('tutorials', 0.7125604152679443),
 ('cpan', 0.7059267163276672)]

In [0]:
# woman + king - man
model.wv.most_similar(positive=['banana', 'orange'], negative=['peach'], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('sugar', 0.5563383102416992),
 ('plantations', 0.5513566732406616),
 ('juice', 0.5482078790664673),
 ('cotton', 0.5477759838104248),
 ('potato', 0.5443365573883057),
 ('wool', 0.5394809246063232),
 ('herring', 0.5300459861755371),
 ('shrimp', 0.5292163491249084),
 ('timber', 0.5209341049194336),
 ('beef', 0.5176421999931335)]

In [0]:
model.wv.similarity("girl", "boy")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.6871714

In [0]:
model.wv.similarity('girl', 'car')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.32335734

In [0]:
model.wv.similarity('apple', 'google')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.22032908

In [0]:
model.wv.similarity("will", "going")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.32553226

## GloVe
- カウントベースの手法
- $R = PQ \approx R'$になるように$P, Q$をSGDで学習させる。
  - $R:$共起行列 $P:$単語特徴行列 $Q: $文脈特徴行列
  - 再構築誤差を最小化させる
- 並列化させることで学習が高速になる
- word2vecよりも精度が高い
- 実装があまりない
- glove-pythonというライブラリがあるhttps://github.com/maciejkula/glove-python
- Pytorch実装 https://nbviewer.jupyter.org/github/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/03.GloVe.ipynb
  

## CNNを用いた分散表現の学習
- 一度に数単語を処理する畳み込みフィルターを学習　→　プーリングして最も重要な概念を表現するベクトルを作成


In [0]:
import collections

from keras.layers import Dense, Dropout, Conv1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
import numpy as np
import codecs

np.random.seed(42)

### 前処理の流れ（英語の場合）
- nltk.word_tokenize等を使って単語を分割
- word_to_id、id_to_wordのディクショナリを作成
- コーパスを単語IDのリストに変換

- 日本語　https://qiita.com/Hironsan/items/2466fe0f344115aff177

In [0]:
VOCAB_SIZE = 5000 #上位5000個のトークンのみを考慮
EMBED_SIZE = 100
NUM_FILTERS = 256
NUM_WORDS = 3 # 各フィルターのサイズ（一度に畳み込む単語の数）
BATCH_SIZE = 64
NUM_EPOCHS = 20

# 語彙の構築
counter = collections.Counter()
with codecs.open("umich-sentiment-train.txt", "r", encoding="utf-8") as fin:
  maxlen = 0
  for line in fin:
    _, sent = line.strip().split("\t")
    try:
      words = [x.lower() for x in nltk.word_tokenize(sent)] # 単語分割
    except LookupError:
      nltk.download("punkt")
      words = [x.lower() for x in nltk.word_tokenize(sent)]
    maxlen = max(maxlen, len(words))
    for word in words:
      counter[word] += 1
    word2index = collections.defaultdict(int) # 初期値が0になるdict
    for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
      word2index[word[0]] = wid + 1
    vocab_sz = len(word2index) + 1
    index2word = {v: k for k, v in word2index.items()}

In [0]:
xs, ys = [], []
with codecs.open("umich-sentiment-train.txt", "r", encoding="utf-8") as fin:
  for line in fin:
    label, sent = line.strip().split("\t")
    ys.append(int(label))
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    wids = [word2index[word] for word in words]
    xs.append(wids)

X = pad_sequences(xs, maxlen=maxlen)
Y = np_utils.to_categorical(ys)

In [0]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=42)

In [0]:
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                   validation_data=(Xtest, Ytest))

Train on 4960 samples, validate on 2126 samples
Epoch 1/20
4960/4960 [==============================] - 1s 252us/step - loss: 0.3100 - acc: 0.8911 - val_loss: 0.0395 - val_acc: 0.9845
Epoch 2/20
4960/4960 [==============================] - 1s 129us/step - loss: 0.0205 - acc: 0.9923 - val_loss: 0.0208 - val_acc: 0.9925
Epoch 3/20
4960/4960 [==============================] - 1s 128us/step - loss: 0.0071 - acc: 0.9980 - val_loss: 0.0175 - val_acc: 0.9939
Epoch 4/20
4960/4960 [==============================] - 1s 131us/step - loss: 0.0030 - acc: 0.9994 - val_loss: 0.0167 - val_acc: 0.9953
Epoch 5/20
4960/4960 [==============================] - 1s 126us/step - loss: 0.0019 - acc: 0.9996 - val_loss: 0.0164 - val_acc: 0.9934
Epoch 6/20
4960/4960 [==============================] - 1s 131us/step - loss: 0.0012 - acc: 0.9998 - val_loss: 0.0160 - val_acc: 0.9929
Epoch 7/20
4960/4960 [==============================] - 1s 135us/step - loss: 0.0013 - acc: 0.9998 - val_loss: 0.0161 - val_acc: 0.9958


## word2vecで学習した分散表現のファインチューニング
- gensimを用いてword2vecモデルを読み込む

いますぐ使える単語埋め込みベクトルのリスト
https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

In [0]:
import collections

from gensim.models import KeyedVectors
from keras.layers import Dense, Dropout, Conv1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
import numpy as np
import codecs

np.random.seed(42)

In [0]:
WORD2VEC_MODEL = 'GoogleNews-vectors-negative300.bin.gz'
VOCAB_SIZE = 5000 #上位5000個のトークンのみを考慮
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3 # 各フィルターのサイズ（一度に畳み込む単語の数）
BATCH_SIZE = 64
NUM_EPOCHS = 10 # 少ないエポック数で学習可能

# 語彙の構築
counter = collections.Counter()
with codecs.open("umich-sentiment-train.txt", "r", encoding="utf-8") as fin:
  maxlen = 0
  for line in fin:
    _, sent = line.strip().split("\t")
    try:
      words = [x.lower() for x in nltk.word_tokenize(sent)] # 単語分割
    except LookupError:
      nltk.download("punkt")
      words = [x.lower() for x in nltk.word_tokenize(sent)]
    maxlen = max(maxlen, len(words))
    for word in words:
      counter[word] += 1
    word2index = collections.defaultdict(int) # 初期値が0になるdict
    for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
      word2index[word[0]] = wid + 1
    vocab_sz = len(word2index) + 1
    index2word = {v: k for k, v in word2index.items()}

In [0]:
xs, ys = [], []
with codecs.open("umich-sentiment-train.txt", "r", encoding="utf-8") as fin:
  for line in fin:
    label, sent = line.strip().split("\t")
    ys.append(int(label))
    words = [x.lower() for x in nltk.word_tokenize(sent)]
    wids = [word2index[word] for word in words]
    xs.append(wids)

X = pad_sequences(xs, maxlen=maxlen)
Y = np_utils.to_categorical(ys)

In [0]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=42)

In [0]:
# word2vecモデルの読み込み
word2vec = KeyedVectors.load_word2vec_format(WORD2VEC_MODEL, binary=True)
embedding_weights = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
  try:
    embedding_weights[index, :] = word2vec[word]
  except KeyError:
    pass

KeyboardInterrupt: ignored

In [0]:
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen, weights=[embedding_weights], trainable=True))
model.add(Dropout(0.2))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                   validation_data=(Xtest, Ytest))

Train on 4960 samples, validate on 2126 samples
Epoch 1/10
4960/4960 [==============================] - 5s 1ms/step - loss: 0.1261 - acc: 0.9565 - val_loss: 0.0229 - val_acc: 0.9915
Epoch 2/10
4960/4960 [==============================] - 1s 169us/step - loss: 0.0117 - acc: 0.9968 - val_loss: 0.0161 - val_acc: 0.9934
Epoch 3/10
4960/4960 [==============================] - 1s 169us/step - loss: 0.0053 - acc: 0.9988 - val_loss: 0.0147 - val_acc: 0.9953
Epoch 4/10
4960/4960 [==============================] - 1s 171us/step - loss: 0.0025 - acc: 0.9996 - val_loss: 0.0134 - val_acc: 0.9958
Epoch 5/10
4960/4960 [==============================] - 1s 173us/step - loss: 0.0015 - acc: 0.9998 - val_loss: 0.0160 - val_acc: 0.9953
Epoch 6/10
4960/4960 [==============================] - 1s 171us/step - loss: 0.0013 - acc: 0.9998 - val_loss: 0.0135 - val_acc: 0.9944
Epoch 7/10
4960/4960 [==============================] - 1s 166us/step - loss: 0.0018 - acc: 0.9996 - val_loss: 0.0121 - val_acc: 0.9962
Ep

## 日本語word2vec
https://qiita.com/makaishi2/items/63b7986f6da93dc55edd

In [0]:
# 夏目漱石「こゝろ」
!wget https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip
!unzip 773_ruby_5968.zip
!ls -l kokoro.txt

--2019-01-10 04:58:41--  https://www.aozora.gr.jp/cards/000148/files/773_ruby_5968.zip
Resolving www.aozora.gr.jp (www.aozora.gr.jp)... 59.106.13.115
Connecting to www.aozora.gr.jp (www.aozora.gr.jp)|59.106.13.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153688 (150K) [application/zip]
Saving to: ‘773_ruby_5968.zip.1’

773_ruby_5968.zip.1 100%[===================>] 150.09K   398KB/s    in 0.4s    

2019-01-10 04:58:43 (398 KB/s) - ‘773_ruby_5968.zip.1’ saved [153688/153688]

Archive:  773_ruby_5968.zip
Made with MacWinZipper™
replace kokoro.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
-rw------- 1 root root 374152 Oct 31  2010 kokoro.txt


In [0]:
import codecs
# ファイル読込み、内部表現化
f = codecs.open('kokoro.txt', "r", "sjis")
text = f.read()
f.close()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])



上　先生と私


一

　私はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これは世間を憚かる遠慮というよりも、その方が私にとって自然だからである。私はその人


、なるべく純白に保存しておいてやりたいのが私の唯一の希望なのですから、私が死んだ後でも、妻が生きている以上は、あなた限りに打ち明けられた私の秘密として、すべてを腹の中にしまっておいて下さい。」






In [0]:
# Janomeのインストール（形態素解析ツール）
!pip install janome

# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

In [0]:
tokens = t.tokenize(text)


In [0]:
print(tokens[13].base_form)
print(tokens[13].part_of_speech)

を
助詞,格助詞,一般,*


In [0]:
# テキストを引数として、形態素解析の結果、名詞・動詞原型のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

In [0]:
ret = extract_words('純白に保存しておいてやりたいのが私の唯一の希望なのですから')
for word in ret:
    print(word)

純白
保存
する
おく
やる
の
私
唯一
希望
の


In [0]:
# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

上
先生
私
一
私
人
先生
呼ぶ
いる


In [0]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [0]:
len(list(model.wv.vocab.keys()))

1103

In [0]:
model.wv['先生']

array([-0.63152605, -0.4034008 ,  0.49717602, -0.8836001 ,  0.0215765 ,
        0.07995319,  0.46121264,  0.47798818, -1.1586196 ,  1.0796764 ,
       -0.8952845 ,  0.7590851 ,  0.17977805, -0.39549515,  0.49009183,
        0.01022144,  0.04203074,  0.33179945,  0.0563493 , -0.2233401 ,
        0.53783077, -0.0076797 , -0.46059194, -0.24684972, -0.48515633,
        1.2881328 ,  0.47631183,  0.37234816, -0.7335672 ,  0.04842857,
        1.0791168 , -0.07906754, -1.168488  ,  0.7914109 ,  0.68883497,
        0.11833351,  0.34808576,  0.05332313,  0.1454438 , -0.44862634,
       -0.5389132 , -0.26333973, -0.03994103, -0.5803978 ,  0.3351799 ,
       -1.607527  , -0.50265354,  0.63783556, -0.8477243 , -0.5878949 ,
        0.00829663, -0.4590744 ,  0.9432696 ,  0.4642421 , -0.01725088,
        0.2717031 ,  0.11149205, -0.423973  ,  0.6660051 , -0.5853358 ,
       -0.7193064 , -0.2939907 , -0.01995611,  0.92258596,  0.30660108,
        0.8585386 ,  0.29963535,  0.6583268 ,  0.06508068, -0.41

In [0]:
model.wv.most_similar('私')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('の', 0.2945314049720764),
 ('相応', 0.27585119009017944),
 ('いる', 0.264004647731781),
 ('与える', 0.2619418501853943),
 ('事情', 0.2493799924850464),
 ('あなた', 0.24713750183582306),
 ('引き取る', 0.23413300514221191),
 ('約束', 0.22421950101852417),
 ('遍', 0.22350457310676575),
 ('限り', 0.22086213529109955)]

In [0]:
model.wv.most_similar(positive=['世間', '罪悪'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('笑', 0.6247190237045288),
 ('性', 0.6137335300445557),
 ('談', 0.6023606657981873),
 ('本当', 0.5751911997795105),
 ('立場', 0.5639123916625977),
 ('活動', 0.5386861562728882),
 ('いらっしゃる', 0.5336498618125916),
 ('昂奮', 0.49236541986465454),
 ('倫理', 0.49098604917526245),
 ('君', 0.482044517993927)]